In [5]:
# Instalação de bibliotecas 

import re
import pandas as pd

import math

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

import time 

from bs4 import BeautifulSoup

## Instala o webdriver conforme a versão do navegador instalado na maquina

service = Service(ChromeDriverManager().install())


In [6]:
## Abre um navegador conforme a url especifica

## Essa variável ou objeto navegador vai ser por onde vamos interagir com as informações da pagina.

navegador = webdriver.Chrome(service = service)  
navegador.get("https://portal.cfm.org.br/busca-medicos/")

time.sleep(3)

In [7]:
## Click para aceitar os cookies
navegador.find_element('xpath', '/html/body/div[1]/div[4]/div[2]/button').click()
time.sleep(3)

In [8]:
## Filtros das buscas

# UF
navegador.find_element('xpath', '//*[@id="uf"]').send_keys('GO')

# Especialidade
navegador.find_element('xpath', '//*[@id="especialidade"]').send_keys('ACUPUNTURA')

## Click para buscar medicos conforme ms parametros inseridos

navegador.find_element('xpath', '//*[@id="buscaForm"]/div/div[4]/div[2]/button').click()
time.sleep(3)

In [9]:
# Loop de paginação

dados_medicos = {
    'nome' : [],
    'crm' : [],
    'dt_inscricao' : [],
    'dt_prim_insc_uf' : [],
    'inscricao' : [],
    'situacao' : [],
    'insc_outro_estado' : [],
    'especialidade' : [],
    'endereco' : [],
    'telefone': []
}


# Lista Inicial com a paginação [1,2,3,4,5,6, ...]
element = navegador.find_elements(By.CLASS_NAME, 'J-paginationjs-page')

# Ultima pagina do loop
ultima_pagina = int(element[-1].text)

# Pagina inicial
pagina = 1

while pagina <= ultima_pagina + 1:

    # A Cada passada do while vou buscar a pagina correspondente na lista que é formada, portanto cada pagina é buscada na lista a cada while.
    element = navegador.find_elements(By.CLASS_NAME, 'J-paginationjs-page')

    for i, e in enumerate(element):

        if int(e.text) == pagina:
            
            element[i].click()

            time.sleep(2)

            site = BeautifulSoup(navegador.page_source, 'html.parser')

            medicos = site.find_all('div', class_=re.compile("resultado-item resultMedico_"))

            for medico in medicos:
                
                info_medico = medico.find_all('div', class_ = 'col')[0].get_text()

                # Capturando o nome do médico
                idx_ini = info_medico.find('CRM:')
                dados_medicos['nome'].append(info_medico[:idx_ini].strip())
                info_medico = info_medico[idx_ini:]


                # Capturando o CRM
                idx_ini = info_medico.find('CRM:')
                idx_fim = info_medico.find('Data de Inscrição:')
                # Posição do primeiro termo + a quantidade de caracteres + até o inicio do segundo termo
                dados_medicos['crm'].append(info_medico[idx_ini + 5:idx_fim].strip())
                info_medico = info_medico[idx_fim:]


                # Capturando a data de inscrição
                idx_ini = info_medico.find('Data de Inscrição:')
                idx_fim = info_medico.find('Primeira inscrição na UF:')
                # Posição do primeiro termo + a quantidade de caracteres até o inicio do segundo termo.
                dados_medicos['dt_inscricao'].append(info_medico[idx_ini + 18:idx_fim].strip())
                info_medico = info_medico[idx_fim:]


                # Capturando a data da primeira inscrição na uf
                idx_ini = info_medico.find('Primeira inscrição na UF:')
                idx_fim = info_medico.find('Inscrição:')
                # Posição do primeiro termo + a quantidade de caracteres até o inicio do segundo termo.
                dados_medicos['dt_prim_insc_uf'].append(info_medico[idx_ini + 25:idx_fim].strip())
                info_medico = info_medico[idx_fim:]


                # Capturando a inscricao
                idx_ini = info_medico.find('Inscrição:')
                idx_fim = info_medico.find('Situação:')
                # Posição do primeiro termo + a quantidade de caracteres até o inicio do segundo termo.
                dados_medicos['inscricao'].append(info_medico[idx_ini + 10:idx_fim].strip())
                info_medico = info_medico[idx_fim:]


                # Capturando a situacao
                idx_ini = info_medico.find('Situação:')
                idx_fim = info_medico.find('Inscrições em outro estado:')
                # Posição do primeiro termo + a quantidade de caracteres até o inicio do segundo termo.
                dados_medicos['situacao'].append(info_medico[idx_ini + 9:idx_fim].strip())
                info_medico = info_medico[idx_fim:]


                # Capturando a inscricao em outro estado
                idx_ini = info_medico.find('Inscrições em outro estado:')
                idx_fim = info_medico.find('Especialidades/Áreas de Atuação:')
                # Posição do primeiro termo + a quantidade de caracteres até o inicio do segundo termo.
                dados_medicos['insc_outro_estado'].append(info_medico[idx_ini + 27:idx_fim].strip())
                info_medico = info_medico[idx_fim:]


                # Capturando a especialidade / aera de atuacao
                idx_ini = info_medico.find('Especialidades/Áreas de Atuação:')
                idx_fim = info_medico.find('Endereço:')
                # Posição do primeiro termo + a quantidade de caracteres até o inicio do segundo termo.
                dados_medicos['especialidade'].append(info_medico[idx_ini + 32:idx_fim].strip())
                info_medico = info_medico[idx_fim:]


                # Capturando a especialidade / aera de atuacao
                idx_ini = info_medico.find('Endereço:')
                idx_fim = info_medico.find('Telefone:')
                # Posição do primeiro termo + a quantidade de caracteres até o inicio do segundo termo.
                dados_medicos['endereco'].append(info_medico[idx_ini + 32:idx_fim].strip())
                info_medico = info_medico[idx_fim:]


                # Capturando o telefone
                idx_ini = info_medico.find('Telefone:')
                # Posição do primeiro termo + a quantidade de caracteres até o inicio do segundo termo.
                dados_medicos['telefone'].append(info_medico[idx_ini + 9:-1].strip())

                
            break

    pagina += 1


In [10]:
# Cria DF
dados_medicos = pd.DataFrame(dados_medicos)

# Gera relatorio
dados_medicos.to_csv('dados_medicos.csv', sep = ';', index = False, encoding= 'utf-8-sig')